# Dataosuces declaration

## IMDB Datasource

In [1]:
from imdb import IMDb
imdb = IMDb()
# [ (x.data['title'], x.data['rating']) for x in imdb.get_top250_movies() ][0:25]

## MovieLens Datasource

In [2]:
import urllib.request
import zipfile

url = 'http://files.grouplens.org/datasets/movielens/ml-20m.zip'
download_path = './data/ml-20m.zip'
extract_path = './data/'

# urllib.request.urlretrieve(url, download_path)

# with zipfile.ZipFile(download_path, 'r') as zip:
#     zip.extractall(extract_path)

In [3]:
datasets = {
    'genome-scores': './data/ml-20m/genome-scores.csv',
    'genome-tags': './data/ml-20m/genome-tags.csv',
    'links': './data/ml-20m/links.csv',
    'movie': './data/ml-20m/movies.csv',
    'ratings': './data/ml-20m/ratings.csv',
    'tags': './data/ml-20m/tags.csv',
}

# Data Profiling

In [4]:
import pandas_profiling
import pandas as pd

def read(dataset):
    print('Reading: ', datasets[dataset])
    return pd.read_csv(datasets[dataset])

In [5]:
from IPython.display import display

# for dataset_k in datasets:
#     df = read(dataset_k)
#     display(df.head())

# Cube creation

In [6]:
links = read('links')
ratings = read('ratings')

Reading:  ./data/ml-20m/links.csv
Reading:  ./data/ml-20m/ratings.csv


## Facts

In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2.0,3.5,1112486027
1,1,29.0,3.5,1112484676
2,1,32.0,3.5,1112484819
3,1,47.0,3.5,1112484727
4,1,50.0,3.5,1112484580


## Dimensions

### Time and Date

In [8]:
from datetime import datetime as dt

mesToTexto = [
    'enero',
    'febrero',
    'marzo',
    'abril',
    'mayo',
    'junio',
    'julio',
    'agosto',
    'septiembre',
    'octubre',
    'noviembre',
    'diciembre',
]

def getDate(ts):
    time = dt.fromtimestamp(ts)
    return {
        'anyo':  time.year,
        'mes':  time.month,
        'dia':  time.day,
        'mes_texto': mesToTexto[time.month-1]
    }

def getHour(ts):
    time = dt.fromtimestamp(ts)
    return {
        'hour':  time.hour,
        'minute':  time.minute,
        'second':  time.second,
    }

(getDate(1112486027), getHour(1112486027))

({'anyo': 2005, 'mes': 4, 'dia': 3, 'mes_texto': 'abril'},
 {'hour': 1, 'minute': 53, 'second': 47})

### Film

In [9]:
def getFilm(movie_id):
    print('Finding movie id')
    imdbId = links.loc[links['movieId'] == movie_id]['imdbId']
    print('Fetching movie')
    movie = imdb.get_movie(imdbId)
    print('Movie fetched')
    return {
        'title': movie.data['title'],
        'year': movie.data['year'],
        '_movieId': int(movie_id),
    }

getFilm(29)

Finding movie id
Fetching movie
Movie fetched


{'title': 'The City of Lost Children', 'year': 1995, '_movieId': 29}

# ETL

In [10]:
datasets = {
    'genome-scores': './data/ml-20m/genome-scores.csv',
    'genome-tags': './data/ml-20m/genome-tags.csv',
    'links': './data/ml-20m/links.csv',
    'movie': './data/ml-20m/movies.csv',
    'ratings': './data/ml-20m/ratings.csv',
    'tags': './data/ml-20m/tags.csv',
}

In [11]:
import sqlite3

db = sqlite3.connect('./data.db')
cursor = db.cursor()

In [12]:
with open('./modeloA.sql') as schema:
    query = schema.read()
    cursor.executescript(query)
    db.commit()

In [13]:
import pandas_profiling
import pandas as pd

def read(dataset):
    print('Reading: ', datasets[dataset])
    return pd.read_csv(datasets[dataset])

In [14]:
def extractRating(*args):
    yield {
        'movieId': args[1],
        'rating': args[2],
        'timestamp': args[3],
    }

In [15]:
def addDate(rating):
    date = getDate(int(rating['timestamp']))
    fecha = (date['dia'], date['mes_texto'], date['anyo'])
    cursor.execute('SELECT * from Fecha WHERE dia=? AND mes=? AND anyo=?', fecha)
    data = cursor.fetchone()
    if (data==None):
        fecha = ("{}-{}-{}".format(date['anyo'], date['mes_texto'], date['dia']),*fecha)
        cursor.execute('INSERT INTO Fecha (fecha, dia, mes, anyo) VALUES (?, ?, ?, ?)', fecha)
        db.commit()
        rating['dateId'] = cursor.lastrowid
    else:
        rating['dateId'] = data[0]   
    yield rating

In [16]:
def addHour(rating):
    hour = getHour(int(rating['timestamp']))
    hora = (hour['hour'],hour['minute'],hour['second'])
    cursor.execute('SELECT * from Hora WHERE hora=? AND minuto=? AND segundo=?', hora)
    data = cursor.fetchone()
    if (data==None):
        hora = (*hora, rating['timestamp'])
        cursor.execute('INSERT INTO Hora (hora, minuto,segundo, timestamp) VALUES (?, ?, ?, ?)', hora)
        db.commit()
        rating['hourId'] = cursor.lastrowid
    else:
        rating['hourId'] = data[0]   
    yield rating

In [17]:
links = read('links')
def addFilm(rating):
    film = getFilm(int(rating['movieId']))
    pelicula = (film['_movieId'], 1)
    cursor.execute('SELECT * from Pelicula WHERE idNatural=? AND 1=?', pelicula)
    data = cursor.fetchone()
    if (data==None):
        pelicula = (film['_movieId'], film['title'], film['year'])
        cursor.execute('INSERT INTO Pelicula (idNatural, titulo, anyoLanzamiento) VALUES (?, ?, ?)', pelicula)
        db.commit()
        rating['filmId'] = cursor.lastrowid
    else:
        rating['filmId'] = data[0]   
    yield rating

Reading:  ./data/ml-20m/links.csv


In [18]:
def load(rating):
    voto = (rating['userId'], rating['filmId'],rating['hourId'],rating['dateId'],rating['rating'])
    cursor.execute('INSERT INTO Voto (clvUsuario, clvPelicula, clvHora, clvFecha, nota) VALUES (?, ?, ?, ?, ?)', voto)
    db.commit()
    yield

In [19]:
def extract():
    with open(datasets['ratings']) as f:
        next(f)
        for line in f:        # create a list of lists
            line = line.split(',')
            rating = {}
            rating['userId'] = line[0]
            rating['movieId'] = line[1]
            rating['rating'] = line[2]
            rating['timestamp'] = line[3].strip('\n')
            yield rating

In [20]:
for a in extract():
    for a in addDate(a):
        print('DATE',a)
        for a in addHour(a):
            print('HOUR',a)
            for a in addFilm(a):
                print('FILM',a)
                for a in load(a):
                    pass

DATE {'userId': '1', 'movieId': '2', 'rating': '3.5', 'timestamp': '1112486027', 'dateId': 1}
HOUR {'userId': '1', 'movieId': '2', 'rating': '3.5', 'timestamp': '1112486027', 'dateId': 1, 'hourId': 1}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '1', 'movieId': '2', 'rating': '3.5', 'timestamp': '1112486027', 'dateId': 1, 'hourId': 1, 'filmId': 1}
DATE {'userId': '1', 'movieId': '29', 'rating': '3.5', 'timestamp': '1112484676', 'dateId': 1}
HOUR {'userId': '1', 'movieId': '29', 'rating': '3.5', 'timestamp': '1112484676', 'dateId': 1, 'hourId': 2}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '1', 'movieId': '29', 'rating': '3.5', 'timestamp': '1112484676', 'dateId': 1, 'hourId': 2, 'filmId': 2}
DATE {'userId': '1', 'movieId': '32', 'rating': '3.5', 'timestamp': '1112484819', 'dateId': 1}
HOUR {'userId': '1', 'movieId': '32', 'rating': '3.5', 'timestamp': '1112484819', 'dateId': 1, 'hourId': 3}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId'

Movie fetched
FILM {'userId': '1', 'movieId': '1036', 'rating': '4.0', 'timestamp': '1112485480', 'dateId': 1, 'hourId': 23, 'filmId': 23}
DATE {'userId': '1', 'movieId': '1079', 'rating': '4.0', 'timestamp': '1094785665', 'dateId': 2}
HOUR {'userId': '1', 'movieId': '1079', 'rating': '4.0', 'timestamp': '1094785665', 'dateId': 2, 'hourId': 24}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '1', 'movieId': '1079', 'rating': '4.0', 'timestamp': '1094785665', 'dateId': 2, 'hourId': 24, 'filmId': 24}
DATE {'userId': '1', 'movieId': '1080', 'rating': '3.5', 'timestamp': '1112485375', 'dateId': 1}
HOUR {'userId': '1', 'movieId': '1080', 'rating': '3.5', 'timestamp': '1112485375', 'dateId': 1, 'hourId': 25}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '1', 'movieId': '1080', 'rating': '3.5', 'timestamp': '1112485375', 'dateId': 1, 'hourId': 25, 'filmId': 25}
DATE {'userId': '1', 'movieId': '1089', 'rating': '3.5', 'timestamp': '1112484669', 'dateId': 1}
HOUR {

Movie fetched
FILM {'userId': '1', 'movieId': '1258', 'rating': '4.0', 'timestamp': '1094785994', 'dateId': 2, 'hourId': 39, 'filmId': 45}
DATE {'userId': '1', 'movieId': '1259', 'rating': '4.0', 'timestamp': '1112485440', 'dateId': 1}
HOUR {'userId': '1', 'movieId': '1259', 'rating': '4.0', 'timestamp': '1112485440', 'dateId': 1, 'hourId': 45}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '1', 'movieId': '1259', 'rating': '4.0', 'timestamp': '1112485440', 'dateId': 1, 'hourId': 45, 'filmId': 46}
DATE {'userId': '1', 'movieId': '1261', 'rating': '3.5', 'timestamp': '1094786113', 'dateId': 2}
HOUR {'userId': '1', 'movieId': '1261', 'rating': '3.5', 'timestamp': '1094786113', 'dateId': 2, 'hourId': 46}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '1', 'movieId': '1261', 'rating': '3.5', 'timestamp': '1094786113', 'dateId': 2, 'hourId': 46, 'filmId': 47}
DATE {'userId': '1', 'movieId': '1262', 'rating': '3.5', 'timestamp': '1112484735', 'dateId': 1}
HOUR {

Movie fetched
FILM {'userId': '1', 'movieId': '1994', 'rating': '3.5', 'timestamp': '1094786087', 'dateId': 2, 'hourId': 66, 'filmId': 67}
DATE {'userId': '1', 'movieId': '1997', 'rating': '3.5', 'timestamp': '1094786034', 'dateId': 2}
HOUR {'userId': '1', 'movieId': '1997', 'rating': '3.5', 'timestamp': '1094786034', 'dateId': 2, 'hourId': 67}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '1', 'movieId': '1997', 'rating': '3.5', 'timestamp': '1094786034', 'dateId': 2, 'hourId': 67, 'filmId': 68}
DATE {'userId': '1', 'movieId': '2021', 'rating': '4.0', 'timestamp': '1112485929', 'dateId': 1}
HOUR {'userId': '1', 'movieId': '2021', 'rating': '4.0', 'timestamp': '1112485929', 'dateId': 1, 'hourId': 68}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '1', 'movieId': '2021', 'rating': '4.0', 'timestamp': '1112485929', 'dateId': 1, 'hourId': 68, 'filmId': 69}
DATE {'userId': '1', 'movieId': '2100', 'rating': '4.0', 'timestamp': '1112485955', 'dateId': 1}
HOUR {

Movie fetched
FILM {'userId': '1', 'movieId': '2716', 'rating': '3.5', 'timestamp': '1094786012', 'dateId': 2, 'hourId': 88, 'filmId': 89}
DATE {'userId': '1', 'movieId': '2761', 'rating': '3.0', 'timestamp': '1112484759', 'dateId': 1}
HOUR {'userId': '1', 'movieId': '2761', 'rating': '3.0', 'timestamp': '1112484759', 'dateId': 1, 'hourId': 89}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '1', 'movieId': '2761', 'rating': '3.0', 'timestamp': '1112484759', 'dateId': 1, 'hourId': 89, 'filmId': 90}
DATE {'userId': '1', 'movieId': '2762', 'rating': '4.0', 'timestamp': '1112485367', 'dateId': 1}
HOUR {'userId': '1', 'movieId': '2762', 'rating': '4.0', 'timestamp': '1112485367', 'dateId': 1, 'hourId': 90}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '1', 'movieId': '2762', 'rating': '4.0', 'timestamp': '1112485367', 'dateId': 1, 'hourId': 90, 'filmId': 91}
DATE {'userId': '1', 'movieId': '2804', 'rating': '3.5', 'timestamp': '1112485414', 'dateId': 1}
HOUR {

...

Movie fetched
FILM {'userId': '3', 'movieId': '904', 'rating': '5.0', 'timestamp': '944917450', 'dateId': 4, 'hourId': 210, 'filmId': 246}
DATE {'userId': '3', 'movieId': '905', 'rating': '3.0', 'timestamp': '944917494', 'dateId': 4}
HOUR {'userId': '3', 'movieId': '905', 'rating': '3.0', 'timestamp': '944917494', 'dateId': 4, 'hourId': 211}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '3', 'movieId': '905', 'rating': '3.0', 'timestamp': '944917494', 'dateId': 4, 'hourId': 211, 'filmId': 247}
DATE {'userId': '3', 'movieId': '919', 'rating': '4.0', 'timestamp': '944917558', 'dateId': 4}
HOUR {'userId': '3', 'movieId': '919', 'rating': '4.0', 'timestamp': '944917558', 'dateId': 4, 'hourId': 212}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '3', 'movieId': '919', 'rating': '4.0', 'timestamp': '944917558', 'dateId': 4, 'hourId': 212, 'filmId': 20}
DATE {'userId': '3', 'movieId': '924', 'rating': '5.0', 'timestamp': '944917812', 'dateId': 4}
HOUR {'userId':

Movie fetched
FILM {'userId': '3', 'movieId': '1196', 'rating': '5.0', 'timestamp': '944917859', 'dateId': 4, 'hourId': 227, 'filmId': 31}
DATE {'userId': '3', 'movieId': '1197', 'rating': '5.0', 'timestamp': '944918215', 'dateId': 4}
HOUR {'userId': '3', 'movieId': '1197', 'rating': '5.0', 'timestamp': '944918215', 'dateId': 4, 'hourId': 203}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '3', 'movieId': '1197', 'rating': '5.0', 'timestamp': '944918215', 'dateId': 4, 'hourId': 203, 'filmId': 262}
DATE {'userId': '3', 'movieId': '1198', 'rating': '5.0', 'timestamp': '944917797', 'dateId': 4}
HOUR {'userId': '3', 'movieId': '1198', 'rating': '5.0', 'timestamp': '944917797', 'dateId': 4, 'hourId': 219}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '3', 'movieId': '1198', 'rating': '5.0', 'timestamp': '944917797', 'dateId': 4, 'hourId': 219, 'filmId': 32}
DATE {'userId': '3', 'movieId': '1199', 'rating': '4.0', 'timestamp': '944919671', 'dateId': 4}
HOUR {'u

Movie fetched
FILM {'userId': '3', 'movieId': '1258', 'rating': '5.0', 'timestamp': '944918103', 'dateId': 4, 'hourId': 220, 'filmId': 45}
DATE {'userId': '3', 'movieId': '1259', 'rating': '5.0', 'timestamp': '944918292', 'dateId': 4}
HOUR {'userId': '3', 'movieId': '1259', 'rating': '5.0', 'timestamp': '944918292', 'dateId': 4, 'hourId': 239}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '3', 'movieId': '1259', 'rating': '5.0', 'timestamp': '944918292', 'dateId': 4, 'hourId': 239, 'filmId': 46}
DATE {'userId': '3', 'movieId': '1261', 'rating': '5.0', 'timestamp': '944918444', 'dateId': 4}
HOUR {'userId': '3', 'movieId': '1261', 'rating': '5.0', 'timestamp': '944918444', 'dateId': 4, 'hourId': 195}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '3', 'movieId': '1261', 'rating': '5.0', 'timestamp': '944918444', 'dateId': 4, 'hourId': 195, 'filmId': 47}
DATE {'userId': '3', 'movieId': '1266', 'rating': '5.0', 'timestamp': '944918631', 'dateId': 4}
HOUR {'us

Movie fetched
FILM {'userId': '3', 'movieId': '1544', 'rating': '4.0', 'timestamp': '945176005', 'dateId': 5, 'hourId': 202, 'filmId': 191}
DATE {'userId': '3', 'movieId': '1584', 'rating': '4.0', 'timestamp': '945175784', 'dateId': 5}
HOUR {'userId': '3', 'movieId': '1584', 'rating': '4.0', 'timestamp': '945175784', 'dateId': 5, 'hourId': 225}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '3', 'movieId': '1584', 'rating': '4.0', 'timestamp': '945175784', 'dateId': 5, 'hourId': 225, 'filmId': 62}
DATE {'userId': '3', 'movieId': '1603', 'rating': '4.0', 'timestamp': '945175870', 'dateId': 5}
HOUR {'userId': '3', 'movieId': '1603', 'rating': '4.0', 'timestamp': '945175870', 'dateId': 5, 'hourId': 197}
Finding movie id
Fetching movie
Movie fetched
FILM {'userId': '3', 'movieId': '1603', 'rating': '4.0', 'timestamp': '945175870', 'dateId': 5, 'hourId': 197, 'filmId': 285}
DATE {'userId': '3', 'movieId': '1653', 'rating': '4.0', 'timestamp': '945175730', 'dateId': 5}
HOUR {'

KeyboardInterrupt: 